In [ ]:
# Install required libraries for imbalanced learning and XGBoost
!pip install -q imbalanced-learn xgboost

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
import warnings
import joblib
from google.colab import files

In [ ]:
# Suppress the specific UserWarning from sklearn
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn.base')

In [ ]:
# Load Dataset
url = 'https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv'
try:
    data = pd.read_csv(url)
except Exception as e:
    print("Error loading data:", e)

# Data Preprocessing
# Handle missing values and separate features/target
for column in ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']:
    data.loc[data[column] == 0, column] = data[column].median()

X = data.drop('Outcome', axis=1)
y = data['Outcome']

# Balance classes using SMOTE
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)


<ipython-input-4-9bae59c97b1b>:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '30.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[data[column] == 0, column] = data[column].median()


In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Check the shapes of the training and testing sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((800, 8), (200, 8), (800,), (200,))

In [ ]:
# Define Models for Comparison
models = {
    'Logistic Regression': LogisticRegression(max_iter=200),
    'Random Forest': RandomForestClassifier(),
    'Support Vector Machine': SVC(probability=True),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    'Neural Network': MLPClassifier(max_iter=300)
}

# Model Training and Evaluation with K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results = {}

for model_name, model in models.items():
    with np.errstate(divide='ignore', invalid='ignore'):
        accuracy = cross_val_score(model, X_train, y_train, cv=kf, scoring='accuracy').mean()
        precision = cross_val_score(model, X_train, y_train, cv=kf, scoring='precision').mean()
        results[model_name] = {'Accuracy': accuracy, 'Precision': precision}


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [07:14:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [07:14:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [07:14:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [07:14:50] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [07:14:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

In [ ]:
# Display results for each model
for model_name, metrics in results.items():
    print(f"\n{model_name} Performance:")
    for metric, score in metrics.items():
        print(f"{metric}: {score:.2f}")



Logistic Regression Performance:
Accuracy: 0.74
Precision: 0.76

Random Forest Performance:
Accuracy: 0.81
Precision: 0.80

Support Vector Machine Performance:
Accuracy: 0.79
Precision: 0.78

K-Nearest Neighbors Performance:
Accuracy: 0.77
Precision: 0.73

XGBoost Performance:
Accuracy: 0.80
Precision: 0.78

Neural Network Performance:
Accuracy: 0.79
Precision: 0.79


In [ ]:
best_model_name = max(results, key=lambda x: (results[x]['Accuracy'], results[x]['Precision']))
best_model_metrics = results[best_model_name]
print(f"\nBest Model: {best_model_name}")
print(f"Accuracy: {best_model_metrics['Accuracy']:.2f}, Precision: {best_model_metrics['Precision']:.2f}")


Best Model: Random Forest
Accuracy: 0.81, Precision: 0.80


In [ ]:
# Test the Best Model on Sample Input Data
# Here, we select the best model based on accuracy and precision
best_model = models[best_model_name]
best_model.fit(X_train, y_train)  # Train on the full training dataset after K-Fold validation

try:
    # Collect user inputs for each feature
    user_input = []
    user_input.append(float(input("Pregnancies (0-17): ")))
    user_input.append(float(input("Glucose (0-199): ")))
    user_input.append(float(input("Blood Pressure (0-122): ")))
    user_input.append(float(input("Skin Thickness (0-99): ")))
    user_input.append(float(input("Insulin (0-846): ")))
    user_input.append(float(input("BMI (0-67): ")))
    user_input.append(float(input("Diabetes Pedigree Function (0.078-2.42): ")))
    user_input.append(float(input("Age (21-81): ")))

    # Convert to numpy array and scale the input
    user_input_scaled = scaler.transform(np.array([user_input]))

    # Make prediction
    with np.errstate(divide='ignore', invalid='ignore'):
        prediction = best_model.predict(user_input_scaled)
        prediction_proba = best_model.predict_proba(user_input_scaled)

    # Output prediction and probability
    print(f"\nSample Prediction (0 = No Diabetes, 1 = Diabetes): {prediction[0]}")
    print(f"Probability of Diabetes: {prediction_proba[0][1]:.2f}")

except ValueError:
    print("Invalid input. Please enter numeric values.")

Pregnancies (0-17): 10
Glucose (0-199): 100
Blood Pressure (0-122): 100
Skin Thickness (0-99): 23
Insulin (0-846): 100
BMI (0-67): 23
Diabetes Pedigree Function (0.078-2.42): 1.45
Age (21-81): 25

Sample Prediction (0 = No Diabetes, 1 = Diabetes): 0
Probability of Diabetes: 0.19


In [ ]:
# After training your best model
joblib.dump(best_model, 'diabetes_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

files.download('diabetes_model.pkl')
files.download('scaler.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Predict the Non-Diabetic Baseline Values to Plot the graph between the inflated values in asked questions that made a person diabetic.

df = pd.read_csv(url)

# Filter non-diabetic data
non_diabetic_data = df[df['Outcome'] == 0]

# Calculate mean values for non-diabetic individuals
baseline_values = non_diabetic_data.mean()

# Display baseline values
baseline_values_dict = baseline_values.to_dict()
print(baseline_values_dict)


{'Pregnancies': 3.298, 'Glucose': 109.98, 'BloodPressure': 68.184, 'SkinThickness': 19.664, 'Insulin': 68.792, 'BMI': 30.3042, 'DiabetesPedigreeFunction': 0.42973400000000006, 'Age': 31.19, 'Outcome': 0.0}
